In [66]:
import json
import os
import re
import pandas as pd

In [62]:
league = 48

In [58]:
with open('data/bootstrap-static.json','r') as f:
    data = json.loads(f.read())

In [82]:
players = {}
for element in data['elements']:
    id = element['id']
    name = element['web_name']
    position = element['element_type']
    players[id] = {
        'name': name,
        'position': position
    }

{'name': 'Cédric', 'position': 2}


,1,2,3,4,5,6,7,8,9,10,...,769,770,771,772,773,774,775,776,777,778
name,Cédric,Leno,Xhaka,Elneny,Holding,Partey,Ødegaard,Tierney,Pépé,White,...,Wady,Doyle,Ballard,Robertson,Charles,O'Reilly,Gilchrist,Gillespie,Amo-Ameyaw,Alonso
position,2,1,3,3,2,3,3,2,3,2,...,1,3,4,3,3,3,2,1,3,3


In [86]:
def list_files(directory):
    files = []
    for root, _, filenames in os.walk(directory):
        for filename in filenames:
            files.append(os.path.join(root, filename))
    return files

In [87]:
player_results = {}

file_list = list_files(f'./data/{league}/entries/')
for file_path in file_list:
    pattern = r"entry_(\d+)_(\d+)_picks"
    match = re.match(pattern, os.path.basename(file_path))

    id = match.group(1)
    week = match.group(2)

    with open(file_path,'r') as f:
        entry_data = json.loads(f.read())

    with open(f'./data/{league}/points/{week}_points.json', 'r') as f:
        points_data = json.loads(f.read())

    picks = entry_data['picks'][:11]
    for pick in picks:        
        element = pick['element']
        points = points_data['elements'][f'{element}']['stats']['total_points']
        player=players[element]
        
        if not element in player_results:
            player_results[element] = {
                'name': player['name'],
                'position': player['position'],
                'points': points,
            }
        else:
            current_points = player_results[element]['points']
            player_results[element]['points'] = current_points+points 

In [98]:
pd.DataFrame.from_dict(player_results, orient='index')\
    .sort_values(by='points', ascending=False)\
    .groupby('position').head(5).sort_values(['position', 'points'])

,name,position,points
81,Raya,1,106
307,Ederson,1,116
281,Alisson,1,125
376,Pope,1,126
478,Sá,1,126
280,Van Dijk,2,127
16,Gabriel,2,137
10,White,2,147
285,Alexander-Arnold,2,156
357,Trippier,2,180
